<img src="https://lh3.googleusercontent.com/pw/AM-JKLVhTn_UySwMdfMwXvoq8l3VN7IkrY9cwtH2YJVMxAlMznUBWC9IpFtgPRIyfAXru4oykkYD-1WjWi0Ao5XgkB9JICvzDBcfn0L_5X2_KOOppsURK5DfSifCC-s7Vx5oQrBUn_BNWn_hfAPdhlVbKQGE=w1097-h235-no?authuser=0" alt="2021年度ゲノム情報解析入門" height="100px" align="middle">

<div align="right"><a href="https://github.com/CropEvol/lecture#section2">実習表ページに戻る</a></div>

# 機械学習 - ゲノム解析での応用例 -

　ここまで、機械学習の様々な手法を学んできましたが、分類や回帰などは現在のゲノム解析でもよく利用されています。

例えば、
- 生物集団のDNA配列と形質データからどの遺伝子が形質に影響を与えているのか予測する
- 遺伝子配列（DNA配列やアミノ酸配列）からパターンを読み取り、遺伝子の機能やタンパク質の構造、種類などを予測する（機能予測、構造予測、トポロジー予測など）
- ゲノム全体にわたる遺伝的変異から作物の収穫量を予測するモデルを作成し、最適な交配系統を考えるなど、育種に活用する（Genomic selection、ゲノム育種など）

などです。

　今回は、実際に活用されている例をいくつか紹介し試していきたいと思います。

* 「遺伝子型データと形質データからどの遺伝子座が形質に影響を与えているか予測する」
* 「アミノ酸配列から膜貫通タンパク質かどうかを分類予測する」
* 「アミノ酸配列からタンパク質の構造を予測する」
* 「遺伝子型データから形質データの値を予測する」

などの解析をやってみましょう。

# 遺伝子型データと形質データからどの遺伝子座が形質に影響を与えているか予測する(量的形質遺伝子座(QTL)を予測する)
**QTL**とは**Quantitative Trait Locus**のことで、日本語では量的形質遺伝子座と言います。

詳しくは遺伝学や遺伝育種学で学ぶことが出来ると思いますが、簡単に言うと葉の長さや籾数などの量的な形質に影響を与えている染色体上のDNA領域のことです。QTLを同定することで交配育種に活かしたり、QTL領域から重要遺伝子の同定を進めて行くことになります。

QTLの同定には混合線形モデルを用いたGBLUP法や機械学習を用いた手法など、**GWAS(Genome wide association study)**と呼ばれる様々なアプローチが考案されています。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2021/images/gwas.png?raw=true" alt="gwas" height="300px">

今回は、少し前に扱った「回帰」の手法を使って、どの遺伝子座が形質に影響を与えているのか予測してみましょう。

イメージとしては以下の様な形です。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2021/images/gwas_linear.png?raw=true" alt="gwas" height="300px">


## 使用するデータセット

今回用いるデータはイネの交配集団のデータになります。
経済形質の1つである籾数(GN=Grain number)と、各系統(サンプル)の遺伝子型のデータが入っています。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2021/images/cross_rice.png?raw=true" alt="cross_rice" height="300px">

遺伝子型の情報としては、染色体上のある位置にコシヒカリという系統と比べて変異が入っている(=1)、コシヒカリと同じ遺伝子型である(=0)というデータになっています。

この様な遺伝子型のデータから籾数を説明するモデルを作り、どの遺伝子座が籾数にとって重要なのかという情報を予測してみます。

In [ ]:
# データセットのダウンロード
!wget -q -O GN_genotype_1157samples.csv https://raw.githubusercontent.com/CropEvol/lecture/master/data/GN_genotype_1157samples.csv

# 読み込み
import pandas as pd
df = pd.read_csv("GN_genotype_1157samples.csv", sep=',', header=0, index_col=0)
df

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2021/images/regression_model2.png?raw=true" alt="gregression_model2was" height="150px">


## 実習内容

 1. 前処理
 2. モデルの作成・学習・評価
 3. 重要な説明変数の確認

### 前処理

　予測モデルを作成する前に、前処理をおこなう必要があります。ここでは、次の3つの前処理をおこなっています。

1. データの準備
  - 説明変数: 各遺伝子座の列
  - 目的変数: 籾数(GrainNumber)の列

1. データの分割
  - トレーニング用のデータとテスト用のデータを用意します。
  - ここでは、トレーニングデータ70%、テストデータ30%に分割。

1. スケーリング
  - 各説明変数の尺度を揃えます。
  - ここでは、各説明変数を標準化。  

　次のセルを実行して、前処理をおこなってください。


In [ ]:
# 前処理
import numpy as np

# データの準備
x = np.array(df.iloc[:, 1:]) # 説明変数
y = np.array(df["GrainNumber"]) # 目的変数

# データ分割: データを「トレーニングデータ70%、テストデータ30%」に分ける
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)

print("training: ", x_train.shape, y_train.shape)
print("test: ", x_test.shape, y_test.shape)

In [ ]:
# スケーリング(標準化)
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
x_train_ss = ss.fit_transform(x_train)
x_test_ss = ss.transform(x_test)

# print(x_train[:10])
# print(x_train_ss[:10])

### モデルの作成・学習・評価

モデルの作成、学習、評価をおこなっていきます。

今回は一旦普通の線形回帰を行ってみましょう。

In [ ]:
# モデル選択＆学習
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(x_train_ss, y_train)

# 係数b、誤差e
b = lr.coef_
e = lr.intercept_
print("Coefficient=", b)
print("Intercept="  , e)

In [ ]:
# モデルの評価: 決定係数R2
r2_train_ss = lr.score(x_train_ss, y_train)  # トレーニングデータ
r2_test_ss = lr.score(x_test_ss, y_test)    # テストデータ
print("training: ", r2_train_ss)
print("test: "    , r2_test_ss)

### 重要な説明変数の確認

約1500のサンプルに対し、遺伝子座の数(説明変数の数)が約6000あるので、当然ですが過学習を引き起こしています。

モデルとしては全然ダメなモデルなのでやり直しなのですが、この時$\beta$の値はどのような値になってるか見てみましょう。

明らかに$\beta$の値としてはおかしな値が推定されていることが分かります。

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# 各説明変数(遺伝子座)の効果(β)を見てみる
chr_snp_nums = df.columns[1:].str[:5].value_counts().sort_index().values
start = 0
chr_label_index = []
for i in range(12):
  end = start + chr_snp_nums[i]
  color = "b" if i % 2 == 0 else "r"
  plt.plot(range(start,end), b[start:end], color=color)
  chr_label_index.append((start+end)/2)
  start = end
plt.ylabel("β value")
plt.xticks(chr_label_index, ["chr{}".format(str(i).zfill(2)) for i in range(1, 13)], rotation=45)
plt.show()

## Lassoを使ってみる

通常の線形回帰では過学習を引き起こしていたので、次は重要な遺伝子座(説明変数)にだけ$\beta$の値を推定する様な、**Lasso**を使ってモデルを作成してみましょう。

### モデルの作成・学習・評価

In [ ]:
# モデル選択＆学習、ここでLassoを指定。alphaは正則化の強さ
from sklearn.linear_model import Lasso
lr = Lasso(alpha=0.3)
lr.fit(x_train_ss, y_train)

# 係数b、誤差e
b = lr.coef_
e = lr.intercept_

# モデルの評価: 決定係数R2
r2_train_ss = lr.score(x_train_ss, y_train)  # トレーニングデータ
r2_test_ss = lr.score(x_test_ss, y_test)    # テストデータ
print("training: ", r2_train_ss)
print("test: "    , r2_test_ss)

### 予測性能の確認(グラフ)
先程よりも良さそうなモデルが出来ました。

作成したモデルでテストデータを予測した結果がどのくらい上手くいっているのか、($R^2$でも確認していますが、)図示して確認してみます。

In [ ]:
# testデータの予測確認
test_predict = lr.predict(x_test_ss)

plt.scatter(y_test, test_predict)
plt.xlabel("Observed")
plt.ylabel("Predicted")
plt.show()

### 重要な説明変数の確認

それなりに未知のデータ(テストデータ)に対しても、ちゃんと予測出来ていそうです。

この様なモデルにおいて、どのような遺伝子座(説明変数)が重要なのか確認してみましょう。

In [ ]:
# 各説明変数(遺伝子座)の効果(β)を見てみる
chr_snp_nums = df.columns[1:].str[:5].value_counts().sort_index().values
start = 0
chr_label_index = []
for i in range(12):
  end = start + chr_snp_nums[i]
  color = "b" if i % 2 == 0 else "r"
  plt.plot(range(start,end), b[start:end], color=color)
  chr_label_index.append((start+end)/2)
  start = end
plt.ylabel("β value")
plt.xticks(chr_label_index, ["chr{}".format(str(i).zfill(2)) for i in range(1, 13)], rotation=45)
plt.show()

　実際にこの7番染色体付近にはFZPと呼ばれる遺伝子が位置しており、籾数に影響を与えていることが知られています。

　この様に、籾数などの経済形質に影響を与えている染色体上の領域を特定することは、生物の遺伝的構造を理解し、ゲノム育種によってより良い品種を作成することにつながっていきます。

　そのため、形質に影響を与えている遺伝子座(QTL)をより正確に特定する手法の開発等、非常に重要な研究課題の1つになっています。

### 実習

　下記のコードを編集して、Lassoの正則化の強さ(`Lasso(alpha=XXX)`のalphaの部分)を変えたり、

Ridge回帰やElasticNetなどを使って結果がどの様に変わるのか見てみて下さい。使い方はモデル選択の時に以下の様にします。

Ridge回帰
```python
from sklearn.linear_model import Ridge
lr = Ridge(alpha=0.3)
lr.fit(x_train_ss, y_train)
```

ElasticNet
```python
from sklearn.linear_model import ElasticNet
lr = ElasticNet(alpha=0.3, l1_ratio=0.5)
lr.fit(x_train_ss, y_train)
```

#### 実習コード

In [ ]:
# モデル選択＆学習、ここでモデルを指定。
from sklearn.linear_model import Lasso
lr = Lasso(alpha=0.3)
lr.fit(x_train_ss, y_train)

# 係数b、誤差e
b = lr.coef_
e = lr.intercept_

# モデルの評価: 決定係数R2
r2_train_ss = lr.score(x_train_ss, y_train)  # トレーニングデータ
r2_test_ss = lr.score(x_test_ss, y_test)    # テストデータ
print("training: ", r2_train_ss)
print("test: "    , r2_test_ss)

# 各説明変数(遺伝子座)の効果(β)を見てみる
chr_snp_nums = df.columns[1:].str[:5].value_counts().sort_index().values
start = 0
chr_label_index = []
for i in range(12):
  end = start + chr_snp_nums[i]
  color = "b" if i % 2 == 0 else "r"
  plt.plot(range(start,end), b[start:end], color=color)
  chr_label_index.append((start+end)/2)
  start = end
plt.ylabel("β value")
plt.xticks(chr_label_index, ["chr{}".format(str(i).zfill(2)) for i in range(1, 13)], rotation=45)
plt.show()

# アミノ酸配列から膜貫通タンパク質を予測する

　遺伝子産物であるタンパク質は、アミノ酸で構成されています。タンパク質の種類や機能を知ることは、そのタンパク質（または、タンパク質をコードする遺伝子）の働きを理解するための第一歩になります。

　例えば、下記に代表されるように、アミノ酸配列から[膜タンパク質（Membrane protein）](https://en.wikipedia.org/wiki/Membrane_protein)かどうかを予測するツールがあります。

- [TMHMM](http://www.cbs.dtu.dk/services/TMHMM/)
- [SOSUI](http://harrier.nagahama-i-bio.ac.jp/sosui/sosui_submit.html)

　ここでは、そのようなツールの簡易版を作成していきます。

<small>※ 上述のTMHMMやSOSUIは、与えられたアミノ酸配列が膜タンパク質かどうかの予測だけでなく、膜貫通領域の部位がどこかも予測できます。今回作成する簡易版ツールは、「膜タンパク質かどうかの予測」のみをおこなうツールです。</small>

## 使用するデータセット

[数値化前のデータセット](https://github.com/CropEvol/lecture/blob/master/data/membrane_dataset.csv): 
- 806行 x 2列
  - 列
    - `label`列: 「膜貫通タンパク質」かどうかが書かれた列
    - `sequence`列: アミノ酸配列が書かれた列
  - 行
    - ヘッダー行（1行）: 列名が書かれた行
    - `membrane`ラベル行（105行）: 「膜貫通タンパク質」ラベル 
    - `non_membrane` ラベル行（700行）: 「膜貫通タンパク質でない」ラベル

　アミノ酸配列のようなデータセットを用いる場合、そのままでは機械学習のデータとして利用できません。まず、　アミノ酸配列をなんらかの数値データに変換する必要があります。




　数値化の一例として、ここでは「アミノ酸配列」を「各アミノ酸の割合」に変換しています。このようなデータセットの数値化は、機械学習の工程でいうと、「前処理」に含まれます。</small>

[数値化後のデータセット](https://github.com/CropEvol/lecture/blob/master/data/membrane_dataset.AAfreq.csv): 
- `label`: 目的変数（正解ラベル）
- `A`-`Y`: 説明変数（20個のアミノ酸の割合）。各アミノ酸の種類を一文字表記にしています（詳しくは[こちら](https://ja.wikipedia.org/wiki/%E3%82%BF%E3%83%B3%E3%83%91%E3%82%AF%E8%B3%AA%E3%82%92%E6%A7%8B%E6%88%90%E3%81%99%E3%82%8B%E3%82%A2%E3%83%9F%E3%83%8E%E9%85%B8)）。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2019/images/AA_to_dataset.png?raw=true" alt="AminoAcids_to_dataset" height="150px">


<small>※ このデータセットのソースは、TMHMM (Sonnhammer et al., 1998) の[トレーニングデータ](http://www.cbs.dtu.dk/~krogh/TMHMM/)です。  
> Sonnhammer, E. L., Von Heijne, G., & Krogh, A. (1998, July). A hidden Markov model for predicting transmembrane helices in protein sequences. In Ismb (Vol. 6, pp. 175-182).</small>

　次のコードセルを実行して、「数値化後のデータセット」をダウンロードしてください。

In [ ]:
# データセットのダウンロード
!wget -q -O membrane_dataset.AAfreq.csv https://raw.githubusercontent.com/CropEvol/lecture/master/data/membrane_dataset.AAfreq.csv

# 読み込み
import pandas as pd
df = pd.read_csv("membrane_dataset.AAfreq.csv", sep=',', header=0)
df

## 実習内容

 1. 前処理
 2. モデルの作成・学習・評価
 3. 予測

### 前処理

　予測モデルを作成する前に、前処理をおこなう必要があります。ここでは、次の3つの前処理をおこなっています。

1. データの準備
  - 説明変数: 各アミノ酸の割合の列
  - 目的変数: `label`列

1. データの分割
  - トレーニング用のデータとテスト用のデータを用意します。
  - ここでは、トレーニングデータ70%、テストデータ30%に分割。

1. スケーリング
  - 各説明変数の尺度を揃えます。
  - ここでは、各説明変数を正規化（0~1の範囲のデータに変換）。  
  <small> ※ 「アミノ酸の割合」をデータとして扱うため、今回この操作はあまり重要ではありません。</small>

1. ラベルの数値変換
  - ラベル（membrane, non_membrane）を数値データ（0,1）に変換します。


　次のセルを実行して、前処理をおこなってください。


In [ ]:
# 前処理
# (1) データの準備
import numpy as np
x = np.array(df.iloc[:, 1:])  # 説明変数（各アミノ酸の割合）
y = np.array(df["label"])   # 目的変数（ラベル）

# (2) データの分割
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, stratify=y, random_state=0)

# (3) 説明変数のスケーリング
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
x_train_mm = mms.fit_transform(x_train) # トレーニングデータ
x_test_mm = mms.transform(x_test) # テストデータ

# (4) 目的変数の数値変換
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train_le = le.fit_transform(y_train) # トレーニングデータ
y_test_le = le.transform(y_test) # テストデータ

# 確認
print(y_test_le)

### モデルの作成・学習・評価

　モデルの作成、学習、評価をおこなっていきます。

```python
# モデルの作成
from sklearn.機能 import 関数 
モデル変数 = 関数(オプション)
# モデルの学習
モデル変数.fit(説明変数, 目的変数) # トレーニングデータ
# モデルの評価
モデル変数.score(説明変数, 目的変数) # トレーニングデータ or テストデータ
```

In [ ]:
# 使用するデータ
X_train, Y_train = x_train_mm, y_train_le  # トレーニングデータ
X_test, Y_test  = x_test_mm, y_test_le   # テストデータ

# モデルの作成（ロジスティック回帰）
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
# モデルの学習
clf.fit(X_train, Y_train)

# モデルの評価（正解率）
print("training data: ", clf.score(X_train, Y_train)) # トレーニングデータ
print("test data: ",    clf.score(X_test, Y_test))  # テストデータ

### 予測

　上で作成したモデルを使って、新しいデータのラベル（membrane/non_membrane）を予測します。

予測したい新しいデータ（4つのタンパク質）: 
- [数値化前](https://github.com/CropEvol/lecture/blob/master/data/membrane_newdata.csv)
- [数値化後](https://github.com/CropEvol/lecture/blob/master/data/membrane_newdata.AAfreq.csv)
  - SUT1_ORYSJ: [Sucrose transport protein SUT1 / Oryza sativa](https://www.uniprot.org/uniprot/Q10R54)
  - SCAM1_ORYSJ: [Secretory carrier-associated membrane protein 1 / Oryza sativa](https://www.uniprot.org/uniprot/Q8H5X5)
  - MADS7_ORYSJ: [MADS-box transcription factor 7 / Oryza sativa](https://www.uniprot.org/uniprot/Q0J466)
  - GAOX2_ORYSJ: [Gibberellin 20 oxidase 2](https://www.uniprot.org/uniprot/Q0JH50)

<small>※ それぞれのアミノ酸配列のデータは、タンパク質データベース[UniProt](https://www.uniprot.org/)から得ています。</small>

　次のコードセルを実行して、4つのタンパク質の数値変換後データを取得してください。

In [ ]:
# 新しいデータのダウンロード
!wget -q -O membrane_newdata.AAfreq.csv https://raw.githubusercontent.com/CropEvol/lecture/master/data/membrane_newdata.AAfreq.csv

# 読み込み
new_df = pd.read_csv("membrane_newdata.AAfreq.csv", sep=',', header=0)
new_df

```python
# 予測
モデル変数.predict(新しいデータ)
```

In [ ]:
# 前処理: データの準備と正規化
x_new = np.array(new_df.iloc[:, 1:])
X_new = mms.transform(x_new)

# 予測
y_pred = clf.predict(X_new)
print(le.inverse_transform(y_pred))  # 数値変換前ラベルに戻して表示

### 実習

　「実習コード」のコードセルに追記して、下記の目標をクリアした予測モデルを構築してください。

> 目標: テストデータの正解率 91%以上


　使用する分類手法は自由に選んでください（下記にリンクをクリックすると、scikit-learnのマニュアルページが開きます）。
  - [ロジスティック回帰](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)
  - [サポートベクトルマシン (SVM)](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html)
  - [決定木](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html)
  - [ランダムフォレスト](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
  - [ニューラルネットワーク](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html)


#### 各モデルのコード例

  - [ロジスティック回帰](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)
  ```python
  clf = LogisticRegression() # デフォルト設定（詳しくは、マニュアルページ参照）
  clf = LogisticRegression(solver="liblinear", max_iter=100)  # 最適化手法: Liblinear
  clf = LogisticRegression(solver="newton-cg", max_iter=100)  # 最適化手法: ニュートン-CG法
  clf = LogisticRegression(solver="sag", max_iter=100)  # 最適化手法: Stochastic Average Gradient
  ```
  - [サポートベクトルマシン (SVM)](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html)
  ```python
  clf = SVC() # デフォルト設定（詳しくは、マニュアルページ参照）
  clf = SVC(kernel="linear") # 線形SVM
  clf = SVC(kernel="poly") # カーネルSVM (Polynomial)
  clf = SVC(kernel="poly") # カーネルSVM (RBF)
  ```
  - [決定木](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html)
  ```python
  clf = DecisionTreeClassifier() # デフォルト設定（詳しくは、マニュアルページ参照）
  
  # 分岐に使う指標: ジニ不純度
  clf = DecisionTreeClassifier(criterion='gini', max_depth=3)  # 枝の深さ: 最大3
  clf = DecisionTreeClassifier(criterion='gini', max_depth=None)  # 枝の深さ: 制限なし
  # 分岐に使う指標: エントロピー（情報量）
  clf = DecisionTreeClassifier(criterion='entropy', max_depth=3)
  clf = DecisionTreeClassifier(criterion='entropy', max_depth=None)
  ```
  - [ランダムフォレスト](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
  ```python
  clf = RandomForestClassifier() # デフォルト設定（詳しくは、マニュアルページ参照）
  clf = RandomForestClassifier(n_estimators=100, max_depth=3)  # 決定木数: 100、枝の深さ: 最大3
  clf = RandomForestClassifier(n_estimators=100, max_depth=None)  # 決定木数: 100、枝の深さ: 上限なし
  ```
  - [ニューラルネットワーク](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html)
  ```python
  clf = MLPClassifier()  # デフォルト設定（詳しくは、マニュアルページ参照）

  # 隠れ層: 1層各100ユニット
  # 最適化: Adam (ADAptive Moment estimation)
  # 学習率: 0.001、トレーニング回数: 1000
  clf = MLPClassifier(hidden_layer_sizes=(100,), solver="adam", learning_rate_init=0.001, max_iter=200)

  # 隠れ層: 3層各3ユニット
  # 最適化: 確率的勾配降下法(SGD)
  # 学習率: 0.01、トレーニング回数: 1000
  clf = MLPClassifier(hidden_layer_sizes=(3,3,3,), solver="sgd", learning_rate_init=0.1, max_iter=1000)
  ```

#### 実習コード

In [ ]:
# 使用するデータ
X_train, Y_train = x_train_mm, y_train_le  # トレーニングデータ
X_test, Y_test  = x_test_mm, y_test_le   # テストデータ
X_new  # 新しいデータ

# ===== 編集エリア: start ===== 
# モデルの準備
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
clf = 

# ===== 編集エリア: end  ===== 

# モデルの学習
clf.fit(X_train, Y_train)

# モデルの評価（正解率）
print("training data: ", clf.score(X_train, Y_train)) # トレーニングデータ
print("test data: ",    clf.score(X_test, Y_test))  # テストデータ

# 予測
# 読み込み
new_df = pd.read_csv("membrane_newdata.AAfreq.csv", sep=',', header=0)
x_new = np.array(new_df.iloc[:, 1:])

# 前処理（正規化）
y_pred = clf.predict(X_new)      # 予測値
print(le.inverse_transform(y_pred))  # 予測値を数値変換前ラベルに戻して表示

#### 解答例

　下記の分類手法のいずれかで、目標を達成したモデルを構築できます。
- サポートベクトルマシン
- ランダムフォレスト

In [ ]:
# 使用するデータ
X_train, Y_train = x_train_mm, y_train_le  # トレーニングデータ
X_test, Y_test  = x_test_mm, y_test_le   # テストデータ
X_new  # 新しいデータ

# ===== 編集エリア: start ===== 
# モデルの準備
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
clf = SVC(kernel="rbf")
clf = RandomForestClassifier(n_estimators=100, max_depth=6, random_state=0)

# ===== 編集エリア: end  ===== 

# モデルの学習
clf.fit(X_train, Y_train)

# モデルの評価（正解率）
print("training data: ", clf.score(X_train, Y_train)) # トレーニングデータ
print("test data: ",    clf.score(X_test, Y_test))  # テストデータ

# 予測
y_pred = clf.predict(X_new)      # 予測値
print(le.inverse_transform(y_pred))  # 予測値を数値変換前ラベルに戻して表示

# アミノ酸配列からタンパク質の構造を予測する

　DNAの塩基配列はまずmRNAに転写され、転写されたmRNAはアミノ酸に翻訳され、翻訳されたアミノ酸配列は立体構造をとることでタンパク質として機能が働きます。この立体構造がどの様な構造になるかによってタンパク質の機能が決まります。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2021/images/centraldogma.png?raw=true" alt="centraldogma" height="300px">

　次世代シーケンサーの登場により、ゲノムデータ(DNAやRNA)を読み取ることが容易となり、
ある生き物がどんな塩基配列のDNAを持っているのか、どんな遺伝子が発現しているのかといったことが簡単に分かる時代になっています。

* 生物のゲノム配列を調べる
  * シーケンサーによりDNAを読み取る
* どの部分がアミノ酸配列に翻訳されているのか調べる
  * RNAを読み取ることで、配列の中でも転写されている部分が分かる
* アミノ酸配列がどの様な立体構造を取るかを調べる
  * ？？？

　しかし、翻訳されたアミノ酸配列がどのような立体構造のタンパク質となるのかを調べることは非常に難しく、実験的に構造を確認するにも時間やコストが大きくかかるため、アミノ酸配列の情報からどの様に立体構造を予測するか、というのは数十年に渡って生物学における最も大きな課題の1つでした。

　近年、このタンパク質の構造予測問題にも**機械学習**が用いられてきており、精度の大幅な向上が進んでいます。

　以前Kaggleという機械学習コンペティションの話を少ししましたが、このアミノ酸配列からの構造予測に関するコンペティションも2年に1度行われており、CASP(Critical Assessment of protein Structure Prediction)と呼ばれています。

　2020年に開催されたCASP14において、Google傘下のDeepMindという会社がAlpha fold2というモデルを開発し、他グループに圧倒的な差をつけて勝利しました。このAlpha fold2と呼ばれるモデルが公開されたことで、非常に高精度なタンパク質の構造予測ができるようになりました。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2021/images/alphafold2_score.png?raw=true" alt="alphafold2_score" height="300px">

GDT値が90以上...実験で確認されたこととほぼ等価

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2021/images/alphafold2.png?raw=true" alt="alphafold2" height="500px">

**Alphafold2のソースコード**

https://github.com/deepmind/alphafold



## Alphafold2お試し版

　Alphafold2を使用するには、数TB単位の高性能なSSDだったり、アミノ酸配列の長さに応じた容量のVRAMを備えたGPUと呼ばれるパーツを持った計算機が必要となります。しかし、Google Colab上で動かすことが出来るお試し版が用意されています。データベースなど完全なものを利用していないので、精度は少し落ちるものの、それでもそれなりに高精度な構造予測をしてくれます。

　時間があれば試して見ましょう。

**Google Colab版 Alphafold2**

https://colab.research.google.com/github/sokrypton/ColabFold/blob/main/AlphaFold2.ipynb


　Alphafold2によるタンパク質の構造予測に限らず、現在は様々な分野で機械学習や情報科学の知識が応用されてきており、今後も情報科学的に生物学の課題解決を試みることは重要なアプローチの１つとなっていくと考えられます。

# 良いモデルを構築するために重要なこと

　実習では、どの分類手法を使うかや、パラメータの値をどうするか、に焦点を当てていましたが、機械学習で一番重要な部分は「前処理: データセットの準備」です。

　今回の場合、
* QTLの同定ではどのような交配集団を用意すべきか
* アミノ酸配列の数値化方法がより良いものは無いか
といった点をもう少し工夫をすることで、予測モデルを改善できるかもしれません。

　今回QTLの同定では「変異が入っているか否か」というデータだけでしたが、実際にはどのような変異が入っているか(ATGCのどの塩基置換なのか、もしくは欠失や挿入・欠失・転移などは起きていないか)も考慮したデータセットにした方が良いかもしれません。

　また、そもそもデータを準備する段階でどのような系統をかけ合わせた交配集団を作ればより正確な同定につながるのか、といったところからしっかり考える必要があります。

　タンパク質の機能予測では、生物学的な知識を応用することが出来るかもしれませんん。一般的に、膜貫通タンパク質には、15-30個の連続した疎水性アミノ酸領域を配列中に保有しています。これは、細胞膜を貫通する領域（膜貫通領域）が、疎水性のアミノ酸で構成されているためです。したがって、このような情報を考慮したデータセットを準備することで、予測モデルの改善が期待できます。

<small>※ 例えば、実習では「個々のアミノ酸の割合」をデータに使いましたが、「連続する複数個のアミノ酸の割合（アミノ酸配列の割合）」を使用することで、予測モデルを改善できるかもしれません。</small>

　**良いデータがなければ、良いモデルはできません。Garbage In Garbage Outです。**

　AIやら人工知能やらと流行りの分野ではありますが、どれだけ難しく最新の手法を使ったからと言ってゴミデータからはゴミみたいな結果しか出ないということです。

　便利な手法であることは間違いないのですが、しっかり計画的につくられた綺麗なデータであれば、そんなに難しい手法は必要ない事が多いのも事実です。

---
# まとめ

　次世代シーケンシング技術の発達により、ゲノムワイドなデータを容易に取得できるようになってきました。アクセス可能なゲノムデータの増加に伴い、機械学習を利用したゲノムデータ解析が"あたり前"の時代になってきています。

　ゲノムデータを解析するための重要なポイントは、ゲノムデータを数値化することです。しかし、闇雲に数値化すれば良いわけではありません。「良いモデル」を構築するためには、適切に情報を数値化して、「良いデータセット」を作る必要があります。  
　データセットができれば、ここ数回の実習で見てきたように、自分で機械学習ベースのデータ解析をおこなうことが可能です。

　今後、自身の研究や仕事でデータ解析をおこなうことがあるかと思います。その際には、自分でコーディングして解析することにチャレンジしてみてください。